In [18]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json
import numpy as np
import datetime
import hvplot.pandas


import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

from alpaca_trade_api.rest import REST, TimeFrame

In [2]:
load_dotenv()
opensea_api_key = os.getenv('OPENSEA_API_KEY')
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Data for Context and Analysis

### Collection Stats

In [3]:
url = "https://api.opensea.io/api/v2/collections/pudgypenguins/stats"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

collection_stats_response_json = response.json()

collection_totals_dict = collection_stats_response_json["total"]

# I had to set this index because there is not index and it was causing an error
# I used ChatGPT to debug this issue
collection_totals_df = pd.DataFrame(collection_totals_dict, index=[0])

collection_totals_df

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol
0,346307.045727,78994,4.383966,4783,103481.760758,11.689999,ETH


## Trait Counts

In [4]:
url = "https://api.opensea.io/api/v2/traits/pudgypenguins"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
traits_json = response.json()

# get the "counts" and not "categories" key
traits_json = traits_json.get("counts", [])

# preview
traits_json

{'Background': {'Beige': 1152,
  'Blue': 1587,
  'Mint': 1389,
  'Red': 755,
  'Tangerine': 1135,
  'Yellow': 873,
  'Pink': 710,
  'Purple': 1282,
  'On The Beach': 1,
  'Supermarket': 1,
  'Green': 1,
  'Trick Or Treating': 1,
  'Underwater': 1},
 'Skin': {'Light Gray': 1189,
  'Mint': 423,
  'Maroon': 731,
  'Normal': 2649,
  'Leopard Pink': 89,
  'Dark Gray': 1329,
  'Baby Pink': 623,
  'Cream': 646,
  'Olive Green': 705,
  'Red': 344,
  'Gold': 44,
  'Leopard Gray': 89,
  'Ice': 22,
  'Navy Blue': 4,
  'Black': 1},
 'Body': {'Vote 4 Pudgy': 203,
  'Kimono Blue': 108,
  'Scarf Blue': 237,
  'Shirt Blue': 290,
  'Fish Lover': 70,
  'Turtleneck Grey': 210,
  'Lei Pink': 168,
  'Bow Tie Blue': 154,
  'Tank Top Yellow': 193,
  'Puffer Orange': 158,
  'Huddle Shirt': 228,
  'Kimono Brown': 268,
  'Lei Assorted': 76,
  'Bronze Medal': 193,
  'Gold Medal': 58,
  'Heart': 102,
  'Turtleneck Green': 254,
  'Scarf Pink': 177,
  'Surfboard Necklace': 166,
  'Swordman': 62,
  'Christmas Sweate

#### All Traits

In [5]:
# df of all traits
# documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
# similar to how OneHotEncoder creates new column names
traits_df_normalized = pd.json_normalize(traits_json)

traits_df_normalized

,Background.Beige,Background.Blue,Background.Mint,Background.Red,Background.Tangerine,Background.Yellow,Background.Pink,Background.Purple,Background.On The Beach,Background.Supermarket,...,Head.Sideways Red,Head.Egg Gold,Head.Flower Crown,Head.Fish Orange,Head.Ice Crown,Head.Hatched Gold,Head.Pineapple Suit,Head.Banana Suit,Head.Normal,Head.Shark Suit
0,1152,1587,1389,755,1135,873,710,1282,1,1,...,90,45,66,44,14,9,1,1,1,1


In [6]:
# visualize

# transpose to one column
traits_df_normalized = traits_df_normalized.transpose()

# sort by most common
traits_df_normalized.sort_values([0], ascending=False, inplace=True)

# find top 5
top_10_traits = traits_df_normalized.head(10)



In [28]:
top_10_traits_graph = top_10_traits.hvplot.bar(
    rot=30,
    title="Pudgy Penguin Top 10 Traits",
    ylabel="Count"
)

hvplot.save(top_10_traits_graph, "traits.html")

top_10_traits_graph

:Bars   [index]   (0)

#### Isolating Traits

In [ ]:
# show all trait keys
traits_json.keys()

dict_keys(['Background', 'Skin', 'Body', 'Face', 'Head'])

In [ ]:
# explode the df
traits_df_exploded =  pd.DataFrame(traits_json)
traits_df_exploded = traits_df_exploded.explode("Background")

# create new dfs for each trait, sorted by values
# if you don't do reset_index(drop=True), it appears as a series
background_trait_df = traits_df_exploded["Background"].dropna().sort_values(ascending=False).reset_index()
skin_trait_df = traits_df_exploded["Skin"].dropna().sort_values(ascending=False).reset_index()
body_trait_df = traits_df_exploded["Body"].dropna().sort_values(ascending=False).reset_index()
face_trait_df = traits_df_exploded["Face"].dropna().sort_values(ascending=False).reset_index()
head_trait_df = traits_df_exploded["Head"].dropna().sort_values(ascending=False).reset_index()

# rename columns and set indices
background_trait_df = background_trait_df.rename(columns={"index":"background", "Background":"count"}).set_index("background")
skin_trait_df = skin_trait_df.rename(columns={"index":"skin", "Skin":"count"}).set_index("skin")
body_trait_df = body_trait_df.rename(columns={"index":"body", "Body":"count"}).set_index("body")
face_trait_df = face_trait_df.rename(columns={"index":"face", "Face":"count"}).set_index("face")
head_trait_df = head_trait_df.rename(columns={"index":"head", "Head":"count"}).set_index("head")

# display trait dataframes
display(background_trait_df, skin_trait_df, body_trait_df, face_trait_df, head_trait_df)

,count
background,
Blue,1587.0
Mint,1389.0
Purple,1282.0
Beige,1152.0
Tangerine,1135.0
Yellow,873.0
Red,755.0
Pink,710.0
On The Beach,1.0


,count
skin,
Normal,2649.0
Dark Gray,1329.0
Light Gray,1189.0
Maroon,731.0
Olive Green,705.0
Cream,646.0
Baby Pink,623.0
Mint,423.0
Red,344.0


,count
body,
Shirt Blue,290.0
Kimono Brown,268.0
Shirt Red,259.0
Turtleneck Green,254.0
Scarf Green,250.0
...,...
Pineapple Suit,1.0
Banana Suit,1.0
Mirrored,1.0


,count
face,
Winking,961.0
Blushing,932.0
Normal,864.0
Circle Glasses,768.0
Cute,588.0
Monocle,496.0
Squad,445.0
Eyepatch,426.0
Cross Eyed,417.0


,count
head,
Headband,357.0
Flat Cap Blue,355.0
Bucket Hat Green,349.0
Backwards Hat Red,294.0
Hat Blue,286.0
Top Hat,282.0
Hat Red,279.0
Beanie Orange,276.0
Flat Cap Tan,274.0


# Data for Model

## 30 Days of Events

In [ ]:
# Set today's date
today = datetime.datetime.now()

# set number of days in past
past_date = today - datetime.timedelta(days=30)

# Convert to unix (starting 1970!)
unix_past_date = int(past_date.timestamp())

# Create blank df to store page data
events_df = pd.DataFrame()

# Initialize next_event variable
next_event = None

while True:
    url = f"https://api.opensea.io/api/v2/events/collection/pudgypenguins?after={unix_past_date}&event_type=sale"
    if next_event:
        # This adds the next event value (and required syntax) to the URL string
        url += f"&next={next_event}"

    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    events_json = response.json()

    # Get the "asset events"
    events_data = events_json.get("asset_events", [])
    
    # Store to df
    page_events_df = pd.json_normalize(events_data)
    
    # Concatenate to master df
    events_df = pd.concat([events_df, page_events_df], ignore_index=True)
    
    # Update next_event for pagination
    next_event = events_json.get("next")

    # If there are no more pages, exit the loop
    if not next_event:
        break

# preview
events_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.image_url,nft.metadata_url,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol
0,sale,,ethereum,,1714693331,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x23e47726002f1b8ead44a772e7cc3535c0fb1c27,0xc498ae0a6deb120e0c7949e9826cae509a5634d120be...,1714693331,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-02T23:42:14.252150,False,False,10750000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1,sale,,ethereum,,1714693259,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xea1974c5cf43aaab83d7471095d887a9d6871ec9,0x820b74efb3b81420ce2796ced714307d1ec01a1e5a84...,1714693259,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-02T23:41:01.904025,False,False,10800000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
2,sale,,ethereum,,1714693247,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa69833b9fda816f1bfc79517e7932e64708df0dd,0x9f2dc01a2340f1cf03671225e317ef83da7977d57120...,1714693247,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-02T23:40:50.022007,False,False,10800000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
3,sale,,ethereum,,1714693163,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xff0b9681ef40fd0992ef52fd5bd94e0fa21c0359,0x399c60307794a478e2bd52ae855d5753da2ed75cfdd8...,1714693163,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-02T23:39:26.274755,False,False,10840000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
4,sale,,ethereum,,1714693151,1,0x8bf23e9225624fc64c75af498afe779351dfa860,0xa69833b9fda816f1bfc79517e7932e64708df0dd,0x17ab022ad03327fa57bf34d95231ae0c0f94fb9ce31e...,1714693151,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-02T23:39:13.767539,False,False,10840000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,sale,,ethereum,,1712114603,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa69833b9fda816f1bfc79517e7932e64708df0dd,0xa58f2208d5020b5ff382913a824c9453435996318504...,1712114603,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-05T02:44:50.210434,False,False,11500000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1385,sale,,ethereum,,1712111939,1,0x66663aa97ca4b2ff0d5df564d7ad3c41d63f0bd0,0x31d5248020ff8e35f5cce912e3d8ec446db51b5c,0x1e60e7d034eb1da125ea3fb61d75b089f0f22ef9b08f...,1712111939,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-09T10:40:14.399128,False,False,11884900000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1386,sale,,ethereum,,1712111675,1,0x66663aa97ca4b2ff0d5df564d7ad3c41d63f0bd0,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x0810b26a6883bd81910c32d1bee333178b93a4dfb196...,1712111675,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-03T02:34:37.303342,False,False,11884900000000000000,0x0000000000000000000000000000000000000000,18,ETH
1387,sale,,ethereum,,1712111543,1,0x66663aa97ca4b2ff0d5df564d7ad3c41d63f0bd0,0x2946939

### Individual NFT Info

#### query parameters from https://docs.opensea.io/reference/get_nft

In [ ]:
# PATH PARAMETERS

# note: nft contract is the same for the whole collection

# identify nft contract (public blockchain address)
# this creates an array
protocol_address = events_df["nft.contract"].unique()

# eliminate the null by getting the max of the array
protocol_address = np.max(protocol_address)

# find unique values for chain (currency)
chain = events_df["chain"].unique().tolist()

# write value without list
chain = chain[0]

# token ID in a list for iteration
token_id_list = events_df["nft.identifier"].drop_duplicates().to_list()

In [ ]:
# parsing features function
# credit: Sean!

def get_features (nft):
    features = {}
    features["Background"] = nft["traits"][0]["value"]
    features["Skin"] = nft["traits"][1]["value"]
    features["Face"] = nft["traits"][2]["value"]
    features["Head"] = nft["traits"][3]["value"]
    features["Body"] = nft["traits"][4]["value"]
    features["Rarity"] = nft["rarity"]["rank"]
    
    return features

In [ ]:
nfts_dict = {}
token_id = token_id_list[0]

for token_id in token_id_list:
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    nfts_json = response.json()
    
    nfts_dict[token_id] = get_features(nfts_json["nft"])

Sean's code if the above does not work because "nft" is missing in one api call.

        nfts_dict = {}
        token_id = token_id_list[0]

        for token_id in token_id_list:
            url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
            headers = {
                "accept": "application/json",
                "x-api-key": opensea_api_key
            }

            response = requests.get(url, headers=headers)

            # Formatting as json
            nfts_json = response.json()
            if 'nft' in nfts_json:
                nfts_dict[token_id] = get_features(nfts_json["nft"])

In [ ]:
nfts_dict

nfts_df = pd.DataFrame(nfts_dict).transpose()

nfts_df

,Background,Skin,Face,Head,Body,Rarity
5223,Durag Blue,Dark Gray,Blushing,Purple,Lei Assorted,5253
5378,Mint,Red,Puffer Green,Football,Party Hat,753
3118,Mint,Baby Pink,Huddle Shirt,Cross Eyed,Hat Red,4402
830,Maroon,Scarf Blue,Cute,Pink,Bucket Hat Green,4690
7810,Normal,Red,Lei Pink,Monocle,Panda Hat,3172
...,...,...,...,...,...,...
547,Mint,Normal,Backwards Hat Red,Clout Goggles,Suit Blue,5485
7039,Vote 4 Pudgy,Mint,Dark Gray,Clout Goggles,Fish Blue,2482
5886,Mint,Backwards Hat Red,Baby Pink,Blushing,Turtleneck Green,7124
1076,Red,Cream,Bath Robe,Beanie Orange,Monocle,3461


In [ ]:
combined_events_df = events_df[["nft.identifier", "closing_date", "payment.quantity"]]

combined_events_df.set_index("nft.identifier", inplace=True)

combined_events_df

,closing_date,payment.quantity
nft.identifier,,
5223,1714693331,10750000000000000000
5378,1714693259,10800000000000000000
3118,1714693247,10800000000000000000
830,1714693163,10840000000000000000
7810,1714693151,10840000000000000000
...,...,...
4555,1712114603,11500000000000000000
6469,1712111939,11884900000000000000
5886,1712111675,11884900000000000000


In [ ]:
combined_df = combined_events_df.merge(nfts_df, how="outer", right_index=True, left_index=True)

combined_df.sample(10)

,closing_date,payment.quantity,Background,Skin,Face,Head,Body,Rarity
nft.identifier,,,,,,,,
3325,1713139787,10750000000000000000,Normal,Tangerine,Aviator,Mohawk Purple,Shark Tooth Necklace,4575
5726,1712527499,12070000000000000000,Beige,Flat Cap Black,Cream,Circle Glasses,Lab Coat,5400
2875,1713581291,10090000000000000000,Winking,Blue,Baby Pink,Huddle Shirt,Camo Helmet,6853
4436,1713379895,10050000000000000000,Light Gray,Purple,Bow Tie Black,Hat Red,Handlebar Bear,4176
752,1712654327,11620000000000000000,Maroon,Mint,Blushing,Kimono White,Camo Helmet,4132
4665,1712649503,11630000000000000000,Durag Blue,Dark Gray,Purple,Cross Eyed,Tank Top Blue,5756
830,1714693163,10840000000000000000,Maroon,Scarf Blue,Cute,Pink,Bucket Hat Green,4690
3852,1712917799,11360000000000000000,Normal,Tangerine,Eyepatch,Overalls,Grizzly Bear Hat,6416
6992,1712506847,11700000000000000000,Maroon,Mint,Handlebar Bear,Top Hat,Poncho,2690


## Encoding Cat Data (Training)

In [ ]:
combined_df.dtypes

closing_date         int64
payment.quantity    object
Background          object
Skin                object
Face                object
Head                object
Body                object
Rarity              object
dtype: object

#### Change Wei to Eth so that we can convert data types.

In [ ]:
# change payment.quantity (Wei) to float
# we did not use int because of the overfloat error - no difference with float (both numerical)
combined_df[["payment.quantity", "Rarity"]] = combined_df[["payment.quantity", "Rarity"]].astype(float)

# check
combined_df.dtypes

closing_date          int64
payment.quantity    float64
Background           object
Skin                 object
Face                 object
Head                 object
Body                 object
Rarity              float64
dtype: object

In [ ]:
# encoding features

# create instance of OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# create a list of columns to encode
cat_columns = ["Background", "Skin", "Face", "Head", "Body"]

# encode columns
encoded = encoder.fit_transform(combined_df[cat_columns])

# create a dataframe with the encoded columns and numerical columns
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_columns))

# concat with numerical columns
encoded_df = pd.concat([combined_df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# drop cat columns
encoded_df.drop(columns=cat_columns, inplace=True)

encoded_df

,closing_date,payment.quantity,Rarity,Background_Baby Pink,Background_Backwards Hat Blue,Background_Beige,Background_Blue,Background_Bow Tie Blue,Background_Bucket Hat Tan,Background_Cute,...,Body_Surfboard Necklace,Body_Swordman,Body_Tank Top Blue,Body_Top Hat,Body_Tribal Necklace,Body_Turtleneck Blue,Body_Turtleneck Green,Body_Turtleneck Pink,Body_Viking Helmet,Body_Villain Mask
0,1713053207,1.120000e+19,8741.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1714367927,1.191000e+19,6938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1714200911,1.188000e+19,7702.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1714144895,1.201000e+19,7702.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1712767427,1.186000e+19,7702.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,1712232575,1.147000e+19,1602.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1385,1714672115,1.130000e+19,5014.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386,1713162587,1.009000e+19,3235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1387,1713139775,1.075000e+19,3235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## ETH-USD Conversion

## Create static CSV file for model.

In [ ]:
encoded_df.to_csv("encoded_data.csv")